
# Exercise XX: Gibbs sampling

In this notebook we show how to use CUQIpy to sample hierarchical Bayesian models using Gibbs sampling.

Gibbs sampling is a Markov chain Monte Carlo (MCMC) method for sampling a joint probability distribution of multiple random variables.
Compared to sampling all variables simultaneously, Gibbs sampling samples the variables sequentially. The sampling of each variable is achieved by sampling from the conditional distribution of that variable given (fixed, previously sampled) values of the other variables.

Gibbs sampling is often an efficient way of sampling from a joint distribution if the conditional distributions are easy to sample
from. On the other hand, if the conditional distributions are highly correlated and/or are difficult to sample from, then Gibbs sampling can be very inefficient. For these reasons, Gibbs sampling is often a double-edged sword, that needs to be used in the right context.


## Learning objectives

Going through this notebook you will learn to:

- Describe the basic idea of Gibbs sampling.
- Define a hierarchical Bayesian model using CUQIpy.
- Define a Gibbs sampling scheme in CUQIpy.
- Run the Gibbs sampler and analyze the results.

## Table of contents

TODO


## Setup
We start by importing the necessary modules



In [ ]:
import sys; sys.path.append('../../CUQIpy/')
import numpy as np
import matplotlib.pyplot as plt
from cuqi.testproblem import Deconvolution1D
from cuqi.distribution import GaussianCov, Gamma, JointDistribution, GMRF, Laplace_diff
from cuqi.sampler import Gibbs, Linear_RTO, Conjugate, UnadjustedLaplaceApproximation, ConjugateApprox, MetropolisHastings

# 1. The basics of Gibbs sampling (WIP)

To begin let us consider a very simple Bayesian model with two random variables, $d$ and $x$.

\begin{align}
d &\sim \mathrm{Gamma}(1, 10^{-4})\\
x &\sim \mathcal{N}(0, d^{-1}).
\end{align}

As we have already seen in the previous notebooks, we can define models like this in CUQIpy as follows:


In [ ]:
d = Gamma(1, 1)
x = GaussianCov(0, lambda d: 1/d)

Our goal is to sample from the joint distribution $p(d, x)$. We can think of this as our target distribution. 

We define the joint distribution in CUQIpy as follows:

In [ ]:
target = JointDistribution(x, d)

print(target)

One idea for sampling $p(d,x)$ is to define the parameter $\theta = [d; x]$ and sample from distribution $p(\theta)$ at the same time. However, this is only possible for simple models, where the joint distribution is easy to sample from.

Gibbs sampling is an alternative approach, where we alternately sample from the distribution of each variable conditioned on the others. That is we would sample from $p(d|x)$ and $p(x|d)$. We can summarize this procedure as repeating the following steps:

\begin{align}
\text{1. draw } d^{(i+1)} &\sim p(d|x^{(i)})\\
\text{2. draw } x^{(i+1)} &\sim p(x|d^{(i+1)})
\end{align}

where $x^{(i)}$ and $d^{(i)}$ are the values of $x$ and $d$ at iteration $i$. It can be shown that samples drawn from the above scheme will be distributed according to $p(d, x)$.

In [ ]:
# A hack to direct sample a distribution (Should maybe be added to cuqi)
class Direct:
    def __init__(self, target):
        self.target = target
    
    def step(self, x=None):
        return self.target.sample()

## Defining a sampling strategy

When the conditional distributions are easy to sample from, Gibbs sampling can be very efficient. In the example above, we can sample from the conditional distributions as follows:

1. Sample $x^{(i+1)}$ from $p(x|d^{(i)})$,
2. Sample $d^{(i+1)}$ from $p(d|x^{(i+1)})\propto L(d|x^{(i+1)})p(d)$,

where $L(d|x):=p(x|d)$ is the likelihood function with respect to $d$.

**For step 1:** Note that we already know that $p(x|d)=\mathcal{N}(0, d^{-1})$. Hence, we can directly sample from $x | d$ by sampling from $\mathcal{N}(0, d^{-1})$.

**For step 2:** For $p(d|x)$ we could use any MCMC sampler to sample with. However, note that the Gaussian and Gamma distributions are conjugate, so we can recognize $p(d|x)$ as a Gamma distribution:

\begin{align*}
p(d|x) &\propto L(d|x)p(d)\\
&= \exp\left(-\frac{d}{2}x^2\right)\frac{\beta^\alpha}{\Gamma(\alpha)}d^{\alpha-1}\exp(-\beta d)\\
&= \mathrm{Gamma}(\alpha+1/2, \beta+\frac{1}{2}x^2)
\end{align*}

where $\alpha=1$ and $\beta=10^{-4}$ are the parameters of the Gamma distribution.

Instead of explicitly defining the Gamma distribution, CUQIpy provides a `Conjugate` sampler, which can be used to automatically sample from conjugate distributions. Similarly we can use the `Direct` sampler to sample from the Gaussian distribution.

We can now define a Gibbs sampling scheme in CUQIpy as follows:

In [ ]:
# Define the sampling strategy
sampling_strategy = {
    'x' : Direct,
    'd' : Conjugate, # Can't do MetropolisHastings because we need new to keep track of proposal scale #34
}

We can then define the Gibbs sampler and sample from the joint distribution as follows:

In [ ]:
# Gibbs sampler
sampler = Gibbs(target, sampling_strategy)

# Sample
samples = sampler.sample(1000, 1000)


And we can visualize the trace plots as follows:

In [ ]:
samples["x"].plot_trace()
samples["d"].plot_trace()

# 2. Gibbs for inverse problems


## 2.1 Deterministic forward model and data
Consider an inverse problem
$$y=Ax,$$
where $A: \mathbb{R}^n \to \mathbb{R}^m$ is the forward model of the inverse problem, $y\in\mathbb{R}^m$ is the data and $x\in \mathbb{R}^n$ is the parameter of interest.

In this case, we assume that the forward model is a convolution operator. We can load this example from the testproblem library of CUQIpy and visualize the true solution (sharp signal) and data (convolved signal).

In [ ]:
# Model and data
A, y_data, probinfo = Deconvolution1D.get_components(phantom='square')

# Get dimension of signal
n = A.domain_dim

# Plot exact solution and observed data
plt.subplot(121)
probinfo.exactSolution.plot()
plt.title('exact solution')

plt.subplot(122)
y_data.plot()
plt.title("Observed data")

# Print problem information
print(probinfo)

## 2.2 Hierarchical Bayesian model

We are now going to define a hierarchical Bayesian model for the inverse problem.

First, we are going to model the prior distribution of the parameter $x$ as a Gaussian Markov random field (GMRF), i.e.

\begin{align*}
\mathbf{x} \mid d &\sim \mathrm{GMRF}(\mathbf{0}, d)
\end{align*}

This is a built-in distribution in CUQIpy, which models the difference between neighboring elements as a Gaussian, see [GMRF](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.GMRF.html#cuqi.distribution.GMRF) for more details.

We do not know a good choice of precision parameter $d$ for this prior, so we make use of `lambda` (anonymous) functions to make $d$ a conditioning variable of the distribution. The distribution is then defined in CUQIpy as follows.


In [ ]:
x = GMRF(np.zeros(n), lambda d: d)

print(x)

The observed data is corrupted by additive Gaussian noise, so we can assume a Gaussian distribution for $y$ and write

$$\mathbf{y} \mid \mathbf{x}, s \sim \mathcal{N}(\mathbf{A} \mathbf{x}, s^{-1}\mathbf{I}_m),$$

where for this example, we pretend also not to know the noise standard deviation $s$.

Similar to the prior distribution, we can define this distribution as follows.

In [ ]:
y = GaussianCov(A@x, lambda s: 1/s)

print(y)

To model the two *hyperparameters* $d$ and $s$, we can use a weakly informative `Gamma` distribution, i.e. a distribution with a wide range of possible values.

\begin{align*}
        d &\sim \mathrm{Gamma}(1, 10^{-4}) \\
        s &\sim \mathrm{Gamma}(1, 10^{-4})
\end{align*}


In [ ]:
d = Gamma(1, 1e-4)
s = Gamma(1, 1e-4)

print(d)
print(s)

In total, we can summarize the hierarchical Bayesian model as follows (avoiding explicitly writing the conditioning variables):

\begin{align*}
        d &\sim \mathrm{Gamma}(1, 10^{-4}) \\
        s &\sim \mathrm{Gamma}(1, 10^{-4}) \\
        \mathbf{x} &\sim \mathrm{GMRF}(\mathbf{0}, d) \\
        \mathbf{y} &\sim \mathcal{N}(\mathbf{A} \mathbf{x}, s^{-1} \mathbf{I}_m)
\end{align*}

which in CUQIpy matches line-by-line:

In [ ]:
d = Gamma(1, 1e-4)
s = Gamma(1, 1e-4)
x = GMRF(np.zeros(n), lambda d: d)
y = GaussianCov(A@x, lambda s: 1/s)

## Posterior distribution

Similarly as in the previous notebook, we now define the posterior distribution by first creating the joint distribution $p(\mathbf{y}, \mathbf{x}, d, s)$ and then conditioning on the data $\mathbf{y}^\mathrm{data}$ to obtain the posterior defined in density form as

\begin{align*}
p(\mathbf{x}, d, s \mid \mathbf{y}^\mathrm{data}) = L(\mathbf{x}, s \mid \mathbf{y}^\mathrm{data}) p(\mathbf{x} \mid d) p(d) p(s).
\end{align*}

This in done in CUQIpy as follows.

In [ ]:
# Create joint distribution
joint = JointDistribution(y, x, d, s)

# Define posterior by conditioning on the data
posterior = joint(y=y_data)

# View the posterior
print(posterior)

Notice that after conditioning on the data, the distribution associated with
$\mathbf{y}$ became a likelihood function and that the posterior is now
a joint distribution of the variables $d$, $l$, $\mathbf{x}$.



## 2.3 Gibbs Sampler

The hierarchical model above has some important properties that we
can exploit to make the sampling more efficient. First, note that
the Gamma distribution are conjugate priors for the precision of
the Gaussian distributions. This means that we can efficiently sample
from $d$ and $l$ conditional on the other variables.

Second, note that the prior distribution of $\mathbf{x}$ is
a Gaussian Markov random field (GMRF) and that the distribution for
$\mathbf{y}$ is also Gaussian with a Linear operator acting
on $\mathbf{x}$ as the mean variable. This means that we can
efficiently sample from $\mathbf{x}$ conditional on the other
variables using the ``Linear_RTO`` sampler.

Taking these two facts into account, we can define a Gibbs sampler
that uses the ``Conjugate`` sampler for $d$ and $l$ and
the ``Linear_RTO`` sampler for $\mathbf{x}$.

This is done in CUQIpy as follows:



In [ ]:
# Define sampling strategy
sampling_strategy = {
    'x': Linear_RTO,
    'd': Conjugate,
    's': Conjugate
}

# Define Gibbs sampler
sampler = Gibbs(posterior, sampling_strategy)

# Run sampler
samples = sampler.sample(Ns=1000, Nb=200)

## Analyze results

After sampling we can inspect the results. The samples are stored
as a dictionary with the variable names as keys. Samples for each 
variable is stored as a CUQIpy Samples object which contains the
many convenience methods for diagnostics and plotting of MCMC samples.



In [ ]:
# Plot credible intervals for the signal
samples['x'].plot_ci(exact=probinfo.exactSolution)

Trace plot for d



In [ ]:
samples['d'].plot_trace(figsize=(8,2))

Trace plot for s



In [ ]:
samples['s'].plot_trace(figsize=(8,2))

Notice that the true noise standard deviation was $0.05$ which translates to a precision of $s=400$. The trace plot for $s$ shows that the sampler has converged to the true value.

# 3 Exploring other priors

Notice that while the sampling went well in the previous example,
the posterior distribution did not match the characteristics of
the exact solution. We can improve this result by switching to a
prior that better matches the exact solution $\mathbf{x}$.

One choice is the Laplace difference prior, which assumes a
Laplace distribution for the differences between neighboring
elements of $\mathbf{x}$. That is,

$$\begin{align}
\mathbf{x} \sim \text{Laplace_diff}(\mathbf{0}, d^{-1}),
\end{align}
$$

which translates to the assumption that $x_i-x_{i-1} \sim \mathrm{Laplace}(0, d^{-1})$ for $i=1, \ldots, n$.

This prior is implemented in CUQIpy as the ``Laplace_diff`` distribution.
To update our model we simply need to replace the ``GMRF`` distribution
with the ``Laplace_diff`` distribution. Note that the Laplace distribution
is defined via a scale parameter, so we invert the parameter $d$.

This laplace distribution and new posterior can be defined as follows:



In [ ]:
# Define new distribution for x
x = Laplace_diff(np.zeros(n), lambda d: 1/d)

# Define new joint distribution with piecewise constant prior
joint_Ld = JointDistribution(d, s, x, y)

# Define new posterior by conditioning on the data
posterior_Ld = joint_Ld(y=y_data)

print(posterior_Ld)

## 3.1 Gibbs Sampler (with Laplace prior)

Using the same approach as ealier we can define a Gibbs sampler
for this new hierarchical model. The only difference is that we
now need to use a different sampler for $\mathbf{x}$ because
the ``Linear_RTO`` sampler only works for Gaussian distributions.

In this case we use the UnadjustedLaplaceApproximation sampler
for $\mathbf{x}$. We also use an approximate Conjugate
sampler for $d$ which approximately samples from the
posterior distribution of $d$ conditional on the other
variables in an efficient manner. For more details see e.g.
[Uribe et al. (2021)](https://arxiv.org/abs/2104.06919).



In [ ]:
# Define sampling strategy
sampling_strategy = {
    'x': UnadjustedLaplaceApproximation,
    'd': ConjugateApprox,
    's': Conjugate
}

# Define Gibbs sampler
sampler_Ld = Gibbs(posterior_Ld, sampling_strategy)

# Run sampler
samples_Ld = sampler_Ld.sample(Ns=1000, Nb=200)

## 3.2 Analyze results

Again we can inspect the results.
Here we notice the posterior distribution matches the exact solution much better.



In [ ]:
# Plot credible intervals for the signal
samples_Ld['x'].plot_ci(exact=probinfo.exactSolution)

In [ ]:
samples_Ld['d'].plot_trace(figsize=(8,2))

In [ ]:
samples_Ld['s'].plot_trace(figsize=(8,2))